In [1]:
from collections import Counter
import sys

## Helpers

In [2]:
def isGoal(state,goalState):
    if state == goalState:
        return True
    return False

In [3]:
def moveZero(state,newZeroIndex):
    state=list(state)
    i=state.index('0')
    state[i],state[newZeroIndex]=state[newZeroIndex],state[i]
   # state=str(state)
    return state

In [4]:
def findNeighbours(state):
    neighbours=[]
    zero=state.find('0')
    if zero > 2:
        neighbours.append(moveZero(state,zero-3))
    if zero < 6:
        neighbours.append(moveZero(state,zero+3))
    if zero == 0 or zero == 1 or zero == 3 or zero == 4 or zero == 6 or zero == 7:
        neighbours.append(moveZero(state,zero+1))
    if zero == 1 or zero == 2 or zero == 4 or zero == 5 or zero == 7 or zero == 8:
        neighbours.append(moveZero(state,zero-1))
    return neighbours

# Algorithms

## BFS

In [5]:
def BFS(state,goal):
    frontier={state:True} ## Making it dict "or map" for optimization
    explored=set()
    parent_map={state:None}
    while frontier:
        currState=next(iter(frontier)) ##getting the first element in the map , same as queue
        frontier.pop(currState)
        explored.add(currState) ##Explored set
        if isGoal(currState,goal):
            return parent_map , currState , explored
        for neighbour in findNeighbours(currState):
            string=""
            neighbour=string.join(neighbour) ## neighbour is a list of charachters , so they are converted to one string
            if neighbour not in frontier and neighbour not in explored:
                frontier[neighbour]=True
                parent_map[neighbour]=currState

## DFS

In [6]:
def DFS(state,goal):
    frontier={state:True}
    explored=set()
    parent_map={state:None}
    while frontier:
        currState=frontier.popitem()[0] 
        explored.add(currState) ##Explored set
        if isGoal(currState,goal):
            return parent_map , currState , explored
        for neighbour in findNeighbours(currState):
            string=""
            neighbour=string.join(neighbour) ## neighbour is a list of charachters , so they are converted to one string
            if neighbour not in frontier and neighbour not in explored:
                frontier[neighbour]=True
                parent_map[neighbour]=currState

## A*

In [7]:
import math
def manhattanDist(x_start,y_start,x_goal,y_goal):
    return abs(x_start-x_goal)+abs(y_start-y_goal)

def euclideanDist(x_start,y_start,x_goal,y_goal):
    return math.sqrt(abs(x_start-x_goal)**2+abs(y_start-y_goal)**2)
    

In [8]:
def heuristic(equation,state,goal):
    state=list(state)
    goal=list(goal)
    H=0
    for i,s in enumerate(state):
        if s=='0':
            i=0
            s=0
        else:
            s=goal.index(s)
        x_start,y_start=int(i/3),i%3
        x_goal,y_goal=int(int(s)/3),int(s)%3
        
        if equation=="manhattan":
            H+=manhattanDist(x_start,y_start,x_goal,y_goal)
        elif equation=="euclidean":
            H+=euclideanDist(x_start,y_start,x_goal,y_goal)
            
    return H

In [9]:
def decrease_key(F,equation,currState,G,goal):
    Fnew=heuristic(equation,currState,goal) + G + 1
    return min(Fnew,F)

In [10]:
def A_star(equation,state,goal):
    frontier={state: heuristic(equation,state,goal)}
    parent_map={state: None}
    explored=set()
    
    while frontier:
        currState=min(frontier,key=frontier.get)
        G=frontier[currState] - heuristic(equation,currState,goal) + 1 #F=G+H ,G=F-H
        explored.add(currState)
        frontier.pop(currState)
        
        if isGoal(currState,goal):
            return parent_map,currState,explored
        
        
        for neighbour in findNeighbours(currState):
            string=""
            neighbour=string.join(neighbour) ## neighbour is a list of charachters , so they are converted to one string
            if neighbour not in frontier and neighbour not in explored:
                frontier[neighbour]=heuristic(equation,neighbour,goal)+G+1
                parent_map[neighbour]=currState
            elif neighbour in frontier:
                frontier[neighbour]=decrease_key(frontier[neighbour],equation,neighbour,G,goal)
    return False

# Checkers and printing

In [11]:
def deliverables(parent_map,explored,currState,startState):
    path_to_goal=[]
    cost=0
    current=currState
    while parent_map[current]:
        cost+=1
        path_to_goal.append(current)
        current=parent_map[current]
    path_to_goal.append(startState)
    return path_to_goal , cost

In [12]:
def loopOverState(state):
    countDect = Counter(state)
    for char, count in countDect.items():
        if (count > 1):
            return False
        if (char =='9'):
            return False
    return True

In [13]:
def checkInput(state):
    if len(state)!=9:
        return False
    if (loopOverState(state)==False):
        return False
    if (state.isnumeric()==False):
        return False
    return True

In [14]:
def checkIfSolvable(state):
    inversions=0
    for i in range(0,9):
        for j in range(i+1,9):
            if int(state[i])>int(state[j]) and int(state[i])!=0 and int(state[j])!=0:
                inversions+=1
    if(inversions%2==0):
        return True
    else:
        return False

In [15]:
def printing(printable,goal):
    if(goal=="path"):
        printable.reverse()
    for i in range(len(printable)):
        for j in range(0,10,3):
            print(" ".join(printable[i][j:j+3]))
        print("======================================")

# Menu

In [16]:
def startGame():
    import time
    startState=input("Enter start state:\n")
    if(checkInput(startState)==False):
        print("Invalid input\n")
    elif(checkIfSolvable(startState)==False):
        print("Puzzle is not solvable\n")
    else:
        algo=input("Please Enter algorithm\n")
        parent_map={}
        explored=set()
        currentState=startState
        t0=time.time()
        if(algo=="BFS"):
            parent_map,currentState,explored=BFS(startState,"012345678")
        elif(algo=="DFS"):
            parent_map,currentState,explored=DFS(startState,"012345678")
        elif(algo=="A*"):
            equ=input("Please enter equation to be used: euclidean or manhattan\n")
            if(equ=="manhattan"):
                parent_map,currentState,explored=A_star("manhattan",startState,"012345678")
            elif(equ=="euclidean"):
                parent_map,currentState,explored=A_star("euclidean",startState,"012345678")
            else:
                print("Wrong input , program will exit\n")
                sys.exit()
        else:
                print("Wrong input , program will exit\n")
                sys.exit()            
        p,c=deliverables(parent_map,explored,currentState,startState)
        t1=time.time()
        time=t1-t0
        print("Path to Goal:\n")
        print("=============\n")
        printing(p,"path")
        print("Explored:\n")
        print("========\n")
        printing(list(explored),"e")     
        print("Time expanded:\n==============\n")
        print(time)
        print("\nCost:\n====\n")
        print(c)

In [18]:
startGame()

Enter start state:
012345678
Please Enter algorithm
A*
Please enter equation to be used: euclidean or manhattan
hamo
Wrong input , program will exit



SystemExit: 